# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [1]:
# #langchain library
# !pip install langchain==0.1.0
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

In [2]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [11]:
from langchain import PromptTemplate

prompt_template = """
    I'm AIT GPT, you can ask me anything related to AIT.
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="I'm AIT GPT, you can ask me anything related to AIT.\n    {context}\n    Question: {question}\n    Answer:")

In [12]:
PROMPT.format(
    context = "AIT is in Pathum Thani, Thailand",
    question = "Where is AIT?"
)

"I'm AIT GPT, you can ask me anything related to AIT.\n    AIT is in Pathum Thani, Thailand\n    Question: Where is AIT?\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code).
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [5]:
# !pip install langchain pymupdf
!pip install pypdf
# !pip install pwd

In [7]:
import os

try:
    import pwd  # Try importing pwd
    home_directory = pwd.getpwuid(os.getuid()).pw_dir
except ImportError:
    # Handle the case where pwd is not available (Windows)
    home_directory = os.path.expanduser('~')


In [9]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = 'dataset/data.pdf'
loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [ ]:
# documents

In [ ]:
len(documents)

13

In [ ]:
documents[1]

Document(page_content="Asian Institute of Technology\n15th for SDG#1 No Poverty\n58th for SDG#17 Partnerships for the Goals\n64th for SDG#14 Life Below Water\n67th for SDG#2 Zero Hunger\n80th for SDG#15 Life on Land\n100 - 200th for SDG#6 Clean Water and Sanitation\n100 - 200th for SDG#11 Sustainable Cities and\nCommunities\n100 - 200th for Overall Impact\nIn the 2020 THE Impact Rankings,[10] AIT ranked globally:\n19th for SDG#1 No Poverty\n86th for SDG#2 Zero Hunger\n100 - 200th for SDG#6 Clean Water and Sanitation\n100 - 200th for SDG#14 Life Below Water\n100 - 200th for SDG#15 Life on Land\n300 - 400th for Overall Impact\nIn 2020 QS Global MBA Rankings,[11] AIT Master of Business Administration (MBA) program ranked 14th in\nAsia and 1st in Thailand. It also ranked 7th in Asia for Diversity and 18th in the world for Return on Investment.\nIn the QS World University Rankings by Subject 2020, AIT ranked 2nd in Thailand in Engineering and\nTechnology,[12] 151-200th in the world in Envir

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [ ]:
doc[1]

Document(page_content='in \nengineering, \nadvanced\ntechnologies, sustainable development, and management and planning.\nIt aims to promote technological change and sustainable development in\nthe Asia-Pacific region, through higher education, research, and\noutreach.[1]\nFounded in 1959 as SEATO Graduate School of Engineering, it\nreceives funding from organizations and governments around the\nworld.\nIn 1967, The Constituent Assembly of Thailand approved legislation\nfor the Charter of the newly named Asian Institute of Technology in\nOctober. The Asian Institute of Technology Enabling Act was\npublished in the Royal Thai Government Gazette in November the\nsame year. AIT became independent of SEATO as an institution of', metadata={'source': 'AIT.pdf', 'file_path': 'AIT.pdf', 'page': 0, 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Chromium', 'producer': 'Skia/PDF m121', 'creationDate': "D:20240320125911+00'00'", 'modDa

In [ ]:
len(doc)

56

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [ ]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

load INSTRUCTOR_Transformer
max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [ ]:
#locate vectorstore
vector_path = 'vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [ ]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [ ]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [ ]:
#ready to use
retriever = vectordb.as_retriever()

In [ ]:
retriever.get_relevant_documents("Asian Institute of Technology")

[Document(page_content="Asian Institute of\nTechnology\nAIT\nFormer\nnames\nSEATO Graduate\nSchool of\nEngineering\nMotto\n'Social Impact with\nInnovation'\nEstablished\n1959\nPresident\nKazuo Yamamoto\nAcademic\nstaff\nAbout 100\nStudents\n1,700+\nAddress\n58 Moo 9, Km42,\nPaholyothin\nHighway, Khlong\nLuang, Pathum\nThani, Thailand\nAffiliations\nIAU, ASAIHL,\nGMSARN,\nProSPER.Net,\nACTU, ANSO\nWebsite\nwww.ait.ac.th (http\ns://www.ait.ac.th/)\nAsian Institute of Technology\nThe Asian Institute of Technology (AIT), founded in 1959, is an\ninternational organization for higher education situated 40km north of\nBangkok, \nThailand. \nIt \nspecializes \nin \nengineering, \nadvanced\ntechnologies, sustainable development, and management and planning.", metadata={'source': 'AIT.pdf', 'file_path': 'AIT.pdf', 'page': 0, 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Chromium', 'producer': 'Skia/PDF m121', 'creationDate': "D:2024

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [ ]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [ ]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [ ]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [ ]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = 'lmsys/fastchat-t5-3b-v1.0'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [ ]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [ ]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [ ]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [ ]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm AIT GPT to answer all the question about the institution.\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7bd32bf3bc40>)), document_variable_name='context')

In [ ]:
query = "Asian Institute of Technology"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm AIT GPT to answer all the question about the institution.
    Asian Institute of
Technology
AIT
Former
names
SEATO Graduate
School of
Engineering
Motto
'Social Impact with
Innovation'
Established
1959
President
Kazuo Yamamoto
Academic
staff
About 100
Students
1,700+
Address
58 Moo 9, Km42,
Paholyothin
Highway, Khlong
Luang, Pathum
Thani, Thailand
Affiliations
IAU, ASAIHL,
GMSARN,
ProSPER.Net,
ACTU, ANSO
Website
www.ait.ac.th (http
s://www.ait.ac.th/)
Asian Institute of Technology
The Asian Institute of Technology (AIT), founded in 1959, is an
international organization for higher education situated 40km north of
Bangkok, 
Thailand. 
It 
specializes 
in 
engineering, 
advanced
technologies, sustainable development, and management and planning.

The campus of Asian Institute of Technology is host to several international and non-governmental organizations
including the Regional R

{'input_documents': [Document(page_content="Asian Institute of\nTechnology\nAIT\nFormer\nnames\nSEATO Graduate\nSchool of\nEngineering\nMotto\n'Social Impact with\nInnovation'\nEstablished\n1959\nPresident\nKazuo Yamamoto\nAcademic\nstaff\nAbout 100\nStudents\n1,700+\nAddress\n58 Moo 9, Km42,\nPaholyothin\nHighway, Khlong\nLuang, Pathum\nThani, Thailand\nAffiliations\nIAU, ASAIHL,\nGMSARN,\nProSPER.Net,\nACTU, ANSO\nWebsite\nwww.ait.ac.th (http\ns://www.ait.ac.th/)\nAsian Institute of Technology\nThe Asian Institute of Technology (AIT), founded in 1959, is an\ninternational organization for higher education situated 40km north of\nBangkok, \nThailand. \nIt \nspecializes \nin \nengineering, \nadvanced\ntechnologies, sustainable development, and management and planning.", metadata={'source': 'AIT.pdf', 'file_path': 'AIT.pdf', 'page': 0, 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Chromium', 'producer': 'Skia/PDF m121', 'cr

In [ ]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm AIT GPT to answer all the question about the institution.\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7bd32bf3bc40>)), document_variable_name='context'), question_generator=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:'), llm=HuggingFacePipeline(pipeline=<transfor

## 5. Chatbot

In [ ]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm AIT GPT to answer all the question about the institution.
    chive/2010/9/Report-from-Swedens-least-known-university/). Retrieved 17 June 2019.
49. "Something for Everyone" (https://www.bangkokpost.com/lifestyle/interview/430220/something-fo
r-everyone). Bangkok Post. Retrieved 17 June 2019.
50. "Condolence- Prof. Thomas H. Evans" (https://www.ait.ac.th/2000/04/condolence/). 25 April
2000. Retrieved 3 September 2018.
51. "AIT People" (https://apps.ait.ac.th/people/). Asian Institute of Technology. Retrieved 14 June
2020.
52. "Dr. Prasarn Trairatvorakul" (https://www.bot.or.th/English/AboutBOT/RolesAndHistory/Governor/
Pages/Prasarn.aspx). Retrieved 22 November 2019.

Name
Degree
Degree
Year
Notability
Notes
Boonsrang
Niumpradit
D. Eng.
1978
Former Thai Army chief
Mao Chi-kuo
M. Eng.
(Community &
Regional
Development)
1975


{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': '<pad>  I  am  AIT  GPT  (Global  Partnership  Team)  to  answer  all  the  question  about  the  institution.\n',
 'source_documents': [Document(page_content='chive/2010/9/Report-from-Swedens-least-known-university/). Retrieved 17 June 2019.\n49. "Something for Everyone" (https://www.bangkokpost.com/lifestyle/interview/430220/something-fo\nr-everyone). Bangkok Post. Retrieved 17 June 2019.\n50. "Condolence- Prof. Thomas H. Evans" (https://www.ait.ac.th/2000/04/condolence/). 25 April\n2000. Retrieved 3 September 2018.\n51. "AIT People" (https://apps.ait.ac.th/people/). Asian Institute of Technology. Retrieved 14 June\n2020.\n52. "Dr. Prasarn Trairatvorakul" (https://www.bot.or.th/English/AboutBOT/RolesAndHistory/Governor/\nPages/Prasarn.aspx). Retrieved 22 November 2019.', metadata={'source': 'AIT.pdf', 'file_path': 'AIT.pdf', 'page': 11, 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject':

In [ ]:
prompt_question = "Asian Institute of Technology"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  AIT  GPT  (Global  Partnership  Team)  to  answer  all  the  question  about  the  institution.\n')]
Follow Up Input: Asian Institute of Technology
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm AIT GPT to answer all the question about the institution.
    Asian Institute of
Technology
AIT
Former
names
SEATO Graduate
School of
Engineering
Motto
'Social Impact with
Innovation'
Established
1959
President
Kazuo Yamamoto
Academic
staff
About 100
Students
1,700+
Address
58 Moo 9, Km42,
Paholyothin
Highway, Khlong
Luang, Pathum
Thani, Tha

{'question': 'Asian Institute of Technology',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  AIT  GPT  (Global  Partnership  Team)  to  answer  all  the  question  about  the  institution.\n')],
 'answer': '<pad>  The  Asian  Institute  of  Technology  (AIT)  is  an  international  organization  for  higher  education  situated  40km  north  of  Bangkok,  Thailand.  It  specializes  in  engineering,  advanced  technologies,  sustainable  development,  and  management  and  planning.\n',
 'source_documents': [Document(page_content="Asian Institute of\nTechnology\nAIT\nFormer\nnames\nSEATO Graduate\nSchool of\nEngineering\nMotto\n'Social Impact with\nInnovation'\nEstablished\n1959\nPresident\nKazuo Yamamoto\nAcademic\nstaff\nAbout 100\nStudents\n1,700+\nAddress\n58 Moo 9, Km42,\nPaholyothin\nHighway, Khlong\nLuang, Pathum\nThani, Thailand\nAffiliations\nIAU, ASAIHL,\nGMSARN,\nProSPER.Net,\nACTU, ANSO\nWebsite\nwww.ait.ac.th (http\ns:

In [ ]:
answer['answer']

'<pad>  The  Asian  Institute  of  Technology  (AIT)  is  an  international  organization  for  higher  education  situated  40km  north  of  Bangkok,  Thailand.  It  specializes  in  engineering,  advanced  technologies,  sustainable  development,  and  management  and  planning.\n'